<a href="https://www.kaggle.com/code/brenodahora/reconhecimento-de-placas-de-licenciamento?scriptVersionId=211237701" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Reconhecimento de placas de licenciamento com YOLO

## Instalação das dependências

- **ultralytics==8.2.66**: Biblioteca utilizada para trabalhar com os modelos YOLO.
- **ipywidgets==8.1.3**: Criar widgets interativos no Jupyter Notebook.
- **pycocotools==2.0.8**: Manipular e avaliar datasets no formato COCO.
- **wandb==0.17.5**: Rastreamento de experimentos e gerenciamento de hiperparâmetros de Machine Learning.
- **paddlepaddle==2.6.1**: Framework para reconhecimento óptico de caracteres (OCR).
- **paddleocr==2.8.1**: Biblioteca para reconhecimento óptico de caracteres (OCR).
- **easyocr==1.7.1**: Biblioteca para OCR.

O parâmetro `-q` reduz a saída de logs durante a instalação.


In [ ]:
!pip install ultralytics==8.2.66 ipywidgets==8.1.3 pycocotools==2.0.8 wandb==0.17.5 paddlepaddle==2.6.1 paddleocr==2.8.1 easyocr==1.7.1 -q

## Importação das bibliotecas

In [ ]:
import easyocr
import wandb
import cv2
import PIL
import os
import re
import pylab
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt

from ultralytics import YOLO
from paddleocr import PaddleOCR
from PIL import Image
from numpy import asarray
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from collections import defaultdict
from tqdm import tqdm

# Tamanho de figuras
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

## Carregamento de credenciais e login no Weights & Biases (W&B)

- Abre o arquivo `secrets.json` contendo as credenciais necessárias.

- Faz login no W&B utilizando a chave de API armazenada em `WB_API_KEY`.

In [ ]:
with open('/kaggle/input/secrets/secrets.json') as f:
    secrets = json.load(f)

wandb.login(key=secrets["WB_API_KEY"])

## Validação e conversão de placas
- Verifica se a sequência de caracteres atende aos padrões de placas veiculares do Brasil.
- Converte os caracteres que não seguem o padrão, com o mapa de conversão quando possível.

In [ ]:
def validate_or_convert_plate(license_plate):
    pattern = re.compile(r'^[A-Z]{3}\d[A-Z]\d{2}$|^[A-Z]{3}\d{4}$')
    if license_plate is None or len(license_plate) != 7 or pattern.match(license_plate):
        return None
    
    dict_char_to_int = {'A' : '1',
                        'B' : '8',
                        'C' : '0',
                        'D' : '0',
                        'E' : '8',
                        'F' : '8',
                        'G' : '0',
                        'H' : '8',
                        'I' : '1',
                        'J' : '1',
                        'K' : '4',
                        'L' : '4',
                        'M' : '4',
                        'N' : '4',
                        'O' : '0',
                        'P' : '9',
                        'Q' : '0',
                        'R' : '8',
                        'S' : '5',
                        'T' : '1',
                        'U' : '0',
                        'V' : '7',
                        'W' : '4',
                        'X' : '2',
                        'Y' : '1',
                        'Z' : '7'}
    
    dict_int_to_char = {'0' : 'O',
                        '1' : 'I',
                        '2' : 'Z',
                        '3' : 'J',
                        '4' : 'A',
                        '5' : 'S',
                        '6' : 'G',
                        '7' : 'T',
                        '8' : 'B',
                        '9' : 'B'}
    
    mapping = {0: dict_int_to_char, 1: dict_int_to_char, 2: dict_int_to_char, 3: dict_char_to_int, 5: dict_char_to_int, 6: dict_char_to_int}
    
    converted_license_plate = list(license_plate)
    for j in [0, 1, 2, 3, 5, 6]:
        if(license_plate[j]) in mapping[j].keys():
            converted_license_plate[j] = mapping[j][license_plate[j]]
        else:
            converted_license_plate[j] = license_plate[j]

    converted_license_plate = ''.join(converted_license_plate)
    
    if str(converted_license_plate) != str(license_plate) and pattern.match(converted_license_plate):
        return converted_license_plate
    else:
        return None

## Agrupamento de bounding boxes de motocicletas
- Agrupa bboxes de caracteres de placas de motocicletas, devido a quebra de linha nesse tipo de placa.

In [ ]:
def vertical_distance_bboxes(box1, box2):
    return abs(box1[0][1] - box2[0][1])

def group_close_detections(detections, max_distance=10):
    grouped_detections = []
    used_indices = set()
    
    for i, (base_box, base_text) in enumerate(detections):
        if i in used_indices:
            continue
        group = [(base_box, base_text)]
        used_indices.add(i)
        
        for j, (box, text) in enumerate(detections):
            if j in used_indices:
                continue
            if vertical_distance_bboxes(base_box, box) <= max_distance:
                group.append((box, text))
                used_indices.add(j)
                
        grouped_detections.append(group)
    
    return grouped_detections

def combine_grouped_texts_and_precisions(grouped_detections):
    combined_text = ''
    total_confidence = 0
    count = 0
    
    for group in grouped_detections:
        for _, (text, confidence) in group:
            combined_text += text
            total_confidence += confidence
            count += 1
    
    avg_confidence = total_confidence / count if count > 0 else 0
    return combined_text, avg_confidence

## Predição de caracteres com PaddleOCR e EasyOCR

In [ ]:
def paddle_predict_plate_chars(result, type_plate='car'):
    pattern = re.compile(r'^[A-Z0-9]{7}$')
    
    detections = []
    for line in result:
        if line is None:
            return None, None
        for detection in line:
            detections.append(detection)

    if type_plate == 'motocycle':
        if detections:
            grouped_detections = group_close_detections(detections)
            combined_texts_precisions = combine_grouped_texts_and_precisions(grouped_detections)
            
            if pattern.match(combined_texts_precisions[0]):
                return combined_texts_precisions
    else:
        for box, (text, confidence) in detections:
            if pattern.match(text):
                return text, confidence

    return None, None

def paddle_get_plate_chars(plate_cropped, paddleocr_reader, type_plate):
    result = paddleocr_reader.ocr(asarray(plate_cropped), cls=True)
    
    if result is None:
        return None, None
    
    predicted, score = paddle_predict_plate_chars(result, type_plate)
    
    converted = validate_or_convert_plate(predicted)
    
    return converted, predicted

def easyocr_get_plate_chars(plate_origin, reader, text_threshold=0.6):
    plate = asarray(plate_origin)
    detections = reader.readtext(plate, paragraph=True, threshold=text_threshold, allowlist='ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')

    for detection in detections:
        bbox, text = detection
        text = text.upper().replace(' ', '')
        
        if len(text) == 7:
            text_conv = validate_or_convert_plate(text)
            if text_conv == text and text_conv is not None:
                return None, text
            else:
                return text_conv, text
            
    return None, None

## Plot de matriz de confusão de caracteres

In [ ]:
def confusion_plot(confusions, ocrname, modelname):
    real_chars = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
    predicted_chars = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
    confusion_matrix = np.zeros((len(real_chars), len(predicted_chars)))
    
    real_indexes = {car: i for i, car in enumerate(real_chars)}
    predicted_indexes = {car: j for j, car in enumerate(predicted_chars)}
    
    for real, subdict in confusions.items():
        if real in real_indexes:
            i = real_indexes[real]
            for previsto, count in subdict.items():
                if previsto in predicted_indexes:
                    j = predicted_indexes[previsto]
                    confusion_matrix[i, j] = count
    
    fig, ax = plt.subplots(figsize=(16.54, 11.69))
    cax = ax.matshow(confusion_matrix, cmap='Blues')
    fig.colorbar(cax)
    ax.set_xticks(np.arange(len(predicted_chars)))
    ax.set_yticks(np.arange(len(real_chars)))
    ax.set_xticklabels(predicted_chars, fontsize=12)
    ax.set_yticklabels(real_chars, fontsize=12)
    ax.xaxis.set_ticks_position('bottom')
    plt.xticks(rotation=0)
    
    for i in range(len(real_chars)):
        for j in range(len(predicted_chars)):
            ax.text(j, i, int(confusion_matrix[i, j]),
                    ha='center', va='center', color='black', fontsize=10)

    plt.xlabel('Caracteres Preditos', fontsize=15)
    plt.ylabel('Caracteres Reais', fontsize=15)
    plt.tight_layout(pad=1.5)
    
    plt.savefig(f'confusion_matrix_{ocrname}_{modelname}.png', dpi=300)
    plt.show()

## Gráficos de acertos de placas

In [ ]:
def create_hit_charts(check_hits):
    models = [hit['model'] for hit in check_hits]

    car_total_detect_plate_local_0 = [hit['results']['car_total_detect_plate_local_0'] for hit in check_hits] 
    motocycle_total_detect_plate_local_0 = [hit['results']['motocycle_total_detect_plate_local_0'] for hit in check_hits]
    car_total_detect_plate_local_1 = [hit['results']['car_total_detect_plate_local_1'] for hit in check_hits]
    motocycle_total_detect_plate_local_1 = [hit['results']['motocycle_total_detect_plate_local_1'] for hit in check_hits]

    car_paddle_locale_0_hits = [hit['results']['car_paddle_locale_0_hits'] for hit in check_hits]
    car_paddle_locale_1_hits = [hit['results']['car_paddle_locale_1_hits'] for hit in check_hits]
    motocycle_paddle_locale_0_hits = [hit['results']['motocycle_paddle_locale_0_hits'] for hit in check_hits]
    motocycle_paddle_locale_1_hits = [hit['results']['motocycle_paddle_locale_1_hits'] for hit in check_hits]

    car_easy_locale_0_hits = [hit['results']['car_easy_locale_0_hits'] for hit in check_hits]
    car_easy_locale_1_hits = [hit['results']['car_easy_locale_1_hits'] for hit in check_hits]
    motocycle_easy_locale_0_hits = [hit['results']['motocycle_easy_locale_0_hits'] for hit in check_hits]
    motocycle_easy_locale_1_hits = [hit['results']['motocycle_easy_locale_1_hits'] for hit in check_hits]

    bar_width = 0.15

    r1 = np.arange(len(models))
    r2 = [x + bar_width + 0.025 for x in r1]
    r3 = [x + bar_width + 0.025 for x in r2]

    plt.figure(figsize=(11.69, 6.62))
    
    plt.bar(r1, car_total_detect_plate_local_0, width=bar_width, color='#52493a', label='Local 1 - Carro detecções',edgecolor='black', linewidth=0.5)
    plt.bar(r1, motocycle_total_detect_plate_local_0, width=bar_width, bottom=car_total_detect_plate_local_0, color='#7c8569', label='Local 1 - Moto detecções',edgecolor='black', linewidth=0.5)
    plt.bar(r1, car_total_detect_plate_local_1, width=bar_width, bottom=[i+j for i,j in zip(car_total_detect_plate_local_0, motocycle_total_detect_plate_local_0)], color='#a4ab80', label='Local 2 - Carro detecções',edgecolor='black', linewidth=0.5)
    plt.bar(r1, motocycle_total_detect_plate_local_1, width=bar_width, bottom=[i+j+k for i,j,k in zip(car_total_detect_plate_local_0, motocycle_total_detect_plate_local_0, car_total_detect_plate_local_1)], color='#e8e0ae', label='Local 2 - Moto detecções',edgecolor='black', linewidth=0.5)
    
    plt.bar(r2, car_paddle_locale_0_hits, width=bar_width, color='#ed804b', label='Local 1 - Carros PaddleOCR',edgecolor='black', linewidth=0.5)
    plt.bar(r2, motocycle_paddle_locale_0_hits, width=bar_width, bottom=car_paddle_locale_0_hits, color='#f2a94e', label='Local 1 - Motos PaddleOCR',edgecolor='black', linewidth=0.5)
    plt.bar(r2, car_paddle_locale_1_hits, width=bar_width, bottom=[i+j for i,j in zip(car_paddle_locale_0_hits, motocycle_paddle_locale_0_hits)], color='#f8d252', label='Local 2 - Carros PaddleOCR',edgecolor='black', linewidth=0.5)
    plt.bar(r2, motocycle_paddle_locale_1_hits, width=bar_width, bottom=[i+j+k for i,j,k in zip(car_paddle_locale_0_hits, motocycle_paddle_locale_0_hits, car_paddle_locale_1_hits)], color='#fffd50', label='Local 2 - Motos PaddleOCR',edgecolor='black', linewidth=0.5)
    
    plt.bar(r3, car_easy_locale_0_hits, width=bar_width, color='#1a4484', label='Local 1 - Carros EasyOCR',edgecolor='black', linewidth=0.5)
    plt.bar(r3, motocycle_easy_locale_0_hits, width=bar_width, bottom=car_easy_locale_0_hits, color='#3276ce', label='Local 1 - Motos EasyOCR',edgecolor='black', linewidth=0.5)
    plt.bar(r3, car_easy_locale_1_hits, width=bar_width, bottom=[i+j for i,j in zip(car_easy_locale_0_hits, motocycle_easy_locale_0_hits)], color='#4fadf9', label='Local 2 - Carros EasyOCR',edgecolor='black', linewidth=0.5)
    plt.bar(r3, motocycle_easy_locale_1_hits, width=bar_width, bottom=[i+j+k for i,j,k in zip(car_easy_locale_0_hits, motocycle_easy_locale_0_hits, car_easy_locale_1_hits)], color='#00FFFF', label='Local 2 - Motos EasyOCR',edgecolor='black', linewidth=0.5)
    
    for i in range(len(r1)):
        total_r1 = car_total_detect_plate_local_0[i] + motocycle_total_detect_plate_local_0[i] + car_total_detect_plate_local_1[i] + motocycle_total_detect_plate_local_1[i]
        plt.text(r1[i], total_r1 + 3, str(total_r1), ha='center', va='bottom', fontsize=10)
        
        total_r2 = car_paddle_locale_0_hits[i] + motocycle_paddle_locale_0_hits[i] + car_paddle_locale_1_hits[i] + motocycle_paddle_locale_1_hits[i]
        plt.text(r2[i], total_r2 + 3, str(total_r2), ha='center', va='bottom', fontsize=10)
        
        total_r3 = car_easy_locale_0_hits[i] + motocycle_easy_locale_0_hits[i] + car_easy_locale_1_hits[i] + motocycle_easy_locale_1_hits[i]
        plt.text(r3[i], total_r3 + 3, str(total_r3), ha='center', va='bottom', fontsize=10)
    
    plt.xlabel('Modelo', fontsize=15)
    plt.ylabel('Placas detectadas/reconhecidas', fontsize=15)
    plt.grid(True, alpha=0.25)

    ax = plt.gca()
    ax.set_axisbelow(True)
    ax.set_ylim(0, 600)
    ax.set_yticks(np.arange(0, 601, 25))
    ax.set_yticklabels(ax.get_yticks(), fontsize=12)
    ax.set_xticks(r1 + bar_width + 0.008)
    ax.set_xticklabels(models, rotation=0, ha='center', fontsize=12)
    
    plt.tight_layout(rect=[0, 0, 0.9, 1])
    plt.legend(fontsize=12, ncol=3)
    plt.savefig('line_chart_fullhit_checks.png', dpi=300)
    
    total_plate_mercosul = [hit['results']['total_plate_mercosul'] for hit in check_hits]
    total_plate_gray = [hit['results']['total_plate_gray'] for hit in check_hits]

    total_paddle_mercosul = [hit['results']['total_paddle_mercosul'] for hit in check_hits]
    total_paddle_gray = [hit['results']['total_paddle_gray'] for hit in check_hits]

    total_easyocr_mercosul = [hit['results']['total_easyocr_mercosul'] for hit in check_hits]
    total_easyocr_gray = [hit['results']['total_easyocr_gray'] for hit in check_hits]

    bar_width = 0.15
    r0 = np.arange(len(models))
    r1 = [x + bar_width + 0.025 for x in r0]
    r2 = [x + bar_width + 0.025 for x in r1]

    plt.figure(figsize=(11.69, 6.62))
    
    plt.bar(r0, total_plate_mercosul, width=bar_width, color='#52493a', label='Real - Mercosul',edgecolor='black', linewidth=0.5)
    plt.bar(r0, total_plate_gray, width=bar_width, bottom=total_plate_mercosul, color='#7c8569', label='Real - Cinza',edgecolor='black', linewidth=0.5)
    
    plt.bar(r1, total_paddle_mercosul, width=bar_width, color='#ed804b', label='PaddleOCR - Mercosul',edgecolor='black', linewidth=0.5)
    plt.bar(r1, total_paddle_gray, width=bar_width, bottom=total_paddle_mercosul, color='#f2a94e', label='PaddleOCR - Cinza',edgecolor='black', linewidth=0.5)    
    
    plt.bar(r2, total_easyocr_mercosul, width=bar_width, color='#1a4484', label='EasyOCR - Mercosul',edgecolor='black', linewidth=0.5)
    plt.bar(r2, total_easyocr_gray, width=bar_width, bottom=total_easyocr_mercosul, color='#3276ce', label='EasyOCR - Cinza',edgecolor='black', linewidth=0.5)
    
    for i in range(len(r1)):
        total_r0 = total_plate_mercosul[i] + total_plate_gray[i]
        plt.text(r0[i], total_r0 + 3, str(total_r0), ha='center', va='bottom', fontsize=10)
        
        total_r1 = total_paddle_gray[i] + total_paddle_mercosul[i]
        plt.text(r1[i], total_r1 + 3, str(total_r1), ha='center', va='bottom', fontsize=10)

        total_r2 = total_easyocr_gray[i] + total_easyocr_mercosul[i]
        plt.text(r2[i], total_r2 + 3, str(total_r2), ha='center', va='bottom', fontsize=10)
    
    plt.xlabel('Modelo', fontsize=15)
    plt.ylabel('Placas detectadas/reconhecidas', fontsize=15)
    plt.grid(True, alpha=0.25)

    ax = plt.gca()
    ax.set_axisbelow(True)
    ax.set_ylim(0, 540)
    ax.set_yticks(np.arange(0, 541, 20))
    ax.set_yticklabels(ax.get_yticks(), fontsize=12)
    ax.set_xticks(r0 + bar_width + 0.008)
    ax.set_xticklabels(models, rotation=0, ha='center', fontsize=12)
    
    plt.tight_layout(rect=[0, 0, 0.9, 1])
    plt.legend(fontsize=12, ncol=3)
    plt.savefig('chart_type_plate_checks.png', dpi=300)

## Gráfico de barras para acertos e conversões de caracteres

In [ ]:
def create_graph_bar(stats, name, model_name):
    total_real = stats["total_real"]
    
    total_char_correct_prediction_easyocr = stats["total_char_correct_prediction_easyocr"]
    total_char_correct_conversion_easyocr = stats["total_char_correct_conversion_easyocr"]
    
    total_char_correct_prediction_paddle = stats["total_char_correct_prediction_paddle"]
    total_char_correct_conversion_paddle = stats["total_char_correct_conversion_paddle"]
    
    confusion_plot(stats["easy_confusions"], 'EasyOCR', model_name)
    confusion_plot(stats["paddle_confusions"], 'PaddleOCR', model_name)
    
    labels = list(total_real.keys())
    values_real = list(total_real.values())
    values_correct_easyocr = list(total_char_correct_prediction_easyocr.values())
    values_conversion_easyocr = list(total_char_correct_conversion_easyocr.values())
    values_correct_paddle = list(total_char_correct_prediction_paddle.values())
    values_conversion_paddle = list(total_char_correct_conversion_paddle.values())

    bar_width = 0.25
    r1 = np.arange(len(labels))
    r2 = [x + bar_width for x in r1]
    r3 = [x + bar_width for x in r2]
    
    fig, ax = plt.subplots(figsize=(11.69, 6.62))

    rect1 = ax.bar(r1, values_real, color='gray', alpha=0.6, width=bar_width, edgecolor=None, label='Real')
    rect2 = ax.bar(r2, values_correct_paddle, color='#FF5A33', alpha=0.7, width=bar_width, edgecolor='gray', label='Predito PaddleOCR')
    rect3 = ax.bar(r2, values_conversion_paddle, color='#ffff00', alpha=0.8, width=bar_width, edgecolor='gray', label='Convertido PaddleOCR', bottom=values_correct_paddle)
    rect4 = ax.bar(r3, values_correct_easyocr, color='#025951', alpha=0.7, width=bar_width, edgecolor='gray', label='Predito EasyOCR')
    rect5 = ax.bar(r3, values_conversion_easyocr, color='#0CF25D', alpha=0.8, width=bar_width, edgecolor='gray', label='Convertido EasyOCR', bottom=values_correct_easyocr)

    ax.set_ylim(0, 200)
    ax.set_yticks(np.arange(0, 201, 10))
    ax.set_yticklabels(ax.get_yticks(), fontsize=12)
    ax.grid(True, axis='y', alpha=0.25)
    ax.set_xlabel('Caractere', fontsize=15)
    ax.set_ylabel('Ocorrências', fontsize=15)
    ax.legend(fontsize=12, loc='upper left')
    ax.set_xticks(r1 + bar_width)
    ax.set_xticklabels(labels, rotation=0, ha='center', fontsize=12)

    print('MODELO: ' + model_name)
    
    fig.tight_layout(rect=[0, 0, 0.9, 1])
    plt.savefig(name + '.png', dpi=300)

## Gráfico de precisão média (*average precision* - AP) e sensibilidade média (*average recall* - AR)

In [ ]:
def generate_apr_graph(evals):
    models = [r['model'] for r in evals]
    
    ap_50_95 = [r['results'][0] for r in evals]
    ar_50_95 = [r['results'][1] for r in evals]

    fig, ax = plt.subplots(figsize=(11.69, 6.62))
    ax.set_axisbelow(True)

    width = 0.35
    x = np.arange(len(models))
    rects1 = ax.bar(x - width/2, ap_50_95, width, label='Precisão média (IoU=0.50:0.95)')
    rects2 = ax.bar(x + width/2, ar_50_95, width, label='Sensibilidade média (IoU=0.50:0.95)')

    ax.set_xlabel('Modelo', fontsize=15)
    ax.set_ylabel('Precisão/Sensibilidade', fontsize=15)
    ax.set_xticks(x)
    ax.set_xticklabels(models, rotation=0, ha='center', fontsize=12)
    ax.legend(fontsize=12)
    ax.set_ylim(0, 1)
    ax.set_yticks(np.arange(0, 1.1, 0.1))
    ax.set_yticklabels([f'{tick:.1f}' for tick in ax.get_yticks()], fontsize=12)    
    ax.grid(True, axis='y', alpha=0.6)

    def autolabel(rects):
        for rect in rects:
            height = rect.get_height()
            ax.annotate('{}'.format(round(height, 2)),
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 3),
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=12)
    
    autolabel(rects1)
    autolabel(rects2)

    fig.tight_layout(rect=[0, 0, 0.9, 1])
    plt.savefig('general_apr.png', dpi=300)

## Processamento COCO para geração de gráficos de AP e AR

In [ ]:
def model_coco_eval(path_results):
    annType = ['segm','bbox','keypoints']
    annType = annType[1]
    print(f"Running demo for *{annType}* results.")

    annFile = "/kaggle/input/datatest/coco_annotations_all_test_frames.json"
    cocoGt=COCO(annFile)
    
    resFile=f"/kaggle/working/{path_results}.json"
    cocoDt=cocoGt.loadRes(resFile)
    
    imgIds=sorted(cocoGt.getImgIds())
    imgIds=imgIds[0:100]
    
    cocoEval = COCOeval(cocoGt,cocoDt,annType)
    cocoEval.params.imgIds  = imgIds
    cocoEval.evaluate()
    cocoEval.accumulate()
    cocoEval.summarize()
    
    return [cocoEval.stats[0], cocoEval.stats[8]]

def load_coco_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data

## Conversão de coordenadas de bounding boxes

In [ ]:
def rescale_to_coco_bbox(bbox, img_width=1920, img_height=1080):
    x_center_norm, y_center_norm, w_norm, h_norm = bbox
    
    x_center_pixel = x_center_norm * img_width
    y_center_pixel = y_center_norm * img_height
    w_pixel = w_norm * img_width
    h_pixel = h_norm * img_height
        
    x_left_pixel = x_center_pixel - (w_pixel / 2)
    y_left_pixel = y_center_pixel - (h_pixel / 2)

    return [x_left_pixel, y_left_pixel, w_pixel, h_pixel]

## Cálculo de intersecção sobre a união (*intersection over union* - IoU)

In [ ]:
def calculate_iou(box1, box2):
    # box format: [x, y, width, height]
    x1_min = box1[0]
    y1_min = box1[1]
    x1_max = box1[0] + box1[2]
    y1_max = box1[1] + box1[3]

    x2_min = box2[0]
    y2_min = box2[1]
    x2_max = box2[0] + box2[2]
    y2_max = box2[1] + box2[3]

    # Calculate intersection
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    if inter_x_min < inter_x_max and inter_y_min < inter_y_max:
        inter_area = (inter_x_max - inter_x_min) * (inter_y_max - inter_y_min)
    else:
        inter_area = 0

    # Calculate union
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area

    # Calculate IoU
    iou = inter_area / union_area if union_area != 0 else 0
    return iou

def check_iou_for_image(data, image_id, box, iou_threshold=0.5):
    for annotation in data['annotations']:
        if annotation['image_id'] == image_id:
            iou = calculate_iou(box, annotation['bbox'])
            if iou >= iou_threshold:
                return True
    return False

def filter_bboxes_by_max_iou(bboxes, confidences, coco_data, image_id, iou_threshold=0.5):
    max_iou_bboxes = defaultdict(lambda: (0, None, 0))  # (IoU, bbox, confidence)

    for i, bbox in enumerate(bboxes):
        bbox_rescaled = rescale_to_coco_bbox(bbox.tolist())

        for annotation in coco_data['annotations']:
            if annotation['image_id'] == image_id:
                iou = calculate_iou(bbox_rescaled, annotation['bbox'])

                if iou >= iou_threshold:
                    coord_key = tuple(annotation['bbox'])

                    if iou > max_iou_bboxes[coord_key][0]:
                        max_iou_bboxes[coord_key] = (iou, bbox_rescaled, confidences[i])

    return [(item[1], item[2]) for item in max_iou_bboxes.values()]

## Contagem de confusões entre a placa real e a placa predita

In [ ]:
def multiple_confusion_counter(real_plates, predictions, confusions=None):
    if confusions is None:
        confusions = defaultdict(lambda: defaultdict(int))
    
    for real, predicted in zip(real_plates, predictions):
        for r, p in zip(real, predicted):
            confusions[r][p] += 1
    
    return confusions

## Estatísticas de acertos e conversões de caracteres

In [ ]:
def all_license_plate_hit_check(license_plates):
    mercosul_pattern = re.compile(r'^[A-Z]{3}\d[A-Z]\d{2}$')

    alphanumeric = {char: 0 for char in "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"}

    char_counters = {
        "real": alphanumeric.copy(),
        "pred_easyocr": alphanumeric.copy(),
        "conv_easyocr": alphanumeric.copy(),
        "pred_paddle": alphanumeric.copy(),
        "conv_paddle": alphanumeric.copy(),
        "correct_easyocr": alphanumeric.copy(),
        "correct_paddle": alphanumeric.copy(),
    }

    total_detect_plate_local_0 = 0
    total_detect_plate_local_1 = 0

    car_easy_locale_0_hits = 0
    car_easy_locale_1_hits = 0
    motocycle_easy_locale_0_hits = 0
    motocycle_easy_locale_1_hits = 0
    
    car_paddle_locale_0_hits = 0
    car_paddle_locale_1_hits = 0
    motocycle_paddle_locale_0_hits = 0
    motocycle_paddle_locale_1_hits = 0
    
    car_total_detect_plate_local_0 = 0
    motocycle_total_detect_plate_local_0 = 0
    car_total_detect_plate_local_1 = 0
    motocycle_total_detect_plate_local_1 = 0
    
    total_easyocr_mercosul = 0
    total_easyocr_gray = 0
    total_paddle_mercosul = 0
    total_paddle_gray = 0
    
    total_plate_gray = 0
    total_plate_mercosul = 0
    
    easy_confusions = None
    paddle_confusions = None
    for entry in license_plates:
        real = entry['real_license_plate']
        locale = entry['locale']
        type_plate = entry['type']
        conv_easyocr = entry['conv_license_plate'][0]
        conv_paddle = entry['conv_license_plate'][1]
        pred_easyocr = entry['pred_license_plate'][0]
        pred_paddle = entry['pred_license_plate'][1]

        if pred_easyocr is not None:
            easy_confusions = multiple_confusion_counter([real], [pred_easyocr], easy_confusions)
        if pred_paddle is not None:
            paddle_confusions = multiple_confusion_counter([real], [pred_paddle], paddle_confusions)

        if locale == 0:
            total_detect_plate_local_0 += 1
            if type_plate == 'car':
                car_total_detect_plate_local_0 += 1
            else:
                motocycle_total_detect_plate_local_0 += 1
        else:
            total_detect_plate_local_1 += 1
            if type_plate == 'car':
                car_total_detect_plate_local_1 += 1
            else:
                motocycle_total_detect_plate_local_1 += 1
        
        length = len(real)
        for j in range(length):
            real_char = real[j]
            
            char_counters['real'][real_char] += 1

            if pred_easyocr is not None:
                if real_char == pred_easyocr[j]:
                    char_counters['pred_easyocr'][real_char] += 1
                    char_counters['correct_easyocr'][real_char] += 1

            if pred_paddle is not None:
                if real_char == pred_paddle[j]:
                    char_counters['pred_paddle'][real_char] += 1
                    char_counters['correct_paddle'][real_char] += 1

            if conv_easyocr is not None and conv_easyocr[j] != pred_easyocr[j]:
                if real_char == conv_easyocr[j]:
                    char_counters['conv_easyocr'][real_char] += 1
                    char_counters['correct_easyocr'][real_char] += 1

            if conv_paddle is not None and conv_paddle[j] != pred_paddle[j]:
                if real_char == conv_paddle[j]:
                    char_counters['conv_paddle'][real_char] += 1
                    char_counters['correct_paddle'][real_char] += 1
        
        if mercosul_pattern.match(real):
            total_plate_mercosul += 1
        else:
            total_plate_gray += 1

        if real == pred_easyocr or real == conv_easyocr:
            if mercosul_pattern.match(real):
                total_easyocr_mercosul += 1
            else:
                total_easyocr_gray += 1
              
            if locale == 0:
                if type_plate == 'car':
                    car_easy_locale_0_hits += 1
                else:
                    motocycle_easy_locale_0_hits += 1
            else:
                if type_plate == 'car':
                    car_easy_locale_1_hits += 1
                else:
                    motocycle_easy_locale_1_hits += 1
        
        if real == pred_paddle or real == conv_paddle:
            if mercosul_pattern.match(real):
                total_paddle_mercosul += 1
            else:
                total_paddle_gray += 1

            if locale == 0:
                if type_plate == 'car':
                    car_paddle_locale_0_hits += 1
                else:
                    motocycle_paddle_locale_0_hits += 1
            else:
                if type_plate == 'car':
                    car_paddle_locale_1_hits += 1
                else:
                    motocycle_paddle_locale_1_hits += 1

    result = {
        "easy_confusions": easy_confusions,
        "paddle_confusions": paddle_confusions,
        "total_real": char_counters['real'],
        "total_correct_easyocr": char_counters['correct_easyocr'],
        "total_correct_paddle": char_counters['correct_paddle'],
        "total_char_correct_prediction_easyocr": char_counters['pred_easyocr'],
        "total_char_correct_conversion_easyocr": char_counters['conv_easyocr'],
        "total_char_correct_prediction_paddle": char_counters['pred_paddle'],
        "total_char_correct_conversion_paddle": char_counters['conv_paddle'],
        "car_easy_locale_0_hits": car_easy_locale_0_hits,
        "car_easy_locale_1_hits": car_easy_locale_1_hits,
        "motocycle_easy_locale_0_hits": motocycle_easy_locale_0_hits,
        "motocycle_easy_locale_1_hits": motocycle_easy_locale_1_hits,
        "car_paddle_locale_0_hits": car_paddle_locale_0_hits,
        "car_paddle_locale_1_hits": car_paddle_locale_1_hits,
        "motocycle_paddle_locale_0_hits": motocycle_paddle_locale_0_hits,
        "motocycle_paddle_locale_1_hits": motocycle_paddle_locale_1_hits,
        "total_detect_plate_local_0": total_detect_plate_local_0,
        "total_detect_plate_local_1": total_detect_plate_local_1,
        "car_total_detect_plate_local_0": car_total_detect_plate_local_0,
        "motocycle_total_detect_plate_local_0": motocycle_total_detect_plate_local_0,
        "car_total_detect_plate_local_1": car_total_detect_plate_local_1,
        "motocycle_total_detect_plate_local_1": motocycle_total_detect_plate_local_1,
        "total_easyocr_mercosul": total_easyocr_mercosul,
        "total_easyocr_gray": total_easyocr_gray,
        "total_paddle_mercosul": total_paddle_mercosul,
        "total_paddle_gray": total_paddle_gray,
        "total_plate_mercosul": total_plate_mercosul,
        "total_plate_gray": total_plate_gray
    }

    return result

## Consulta a base de dados de placas para validar os resultados

In [ ]:
def get_frame_data(frame_id: str, database: pd.DataFrame) -> dict:
    if frame_id in database['frame_id'].values:
        line = database[database['frame_id'] == frame_id]
        license_plate_values = line['license_plate'].values[0]
        locale = int(line['locale'].values[0])
        type_value = line['type'].values[0]

        plates = license_plate_values if isinstance(license_plate_values, str) else ''
        count = len(plates.split(',')) if plates else 0
        type_plate = type_value if isinstance(type_value, str) else ''

        return { "plates": plates, "count": count, "locale": locale, "type": type_plate}

    return { "plates": '', "count": 0, "locale": -1, "type": '' }

## Retorna a placa com maior similaridade (maior score) em relação a placa real

In [ ]:
def similarity_score(plate1, plate2):
    if plate1 is None or plate2 is None:
        return 0
    
    return sum(1 for a, b in zip(plate1, plate2) if a == b)

def get_single_real_plate(real_license_plates, conv_easyocr, pred_easyocr, conv_paddleocr, pred_paddleocr):
    best_match = None
    best_score = -1

    for real_plate in real_license_plates:
        score_conv_easyocr = similarity_score(real_plate, conv_easyocr) if conv_easyocr is not None else 0
        score_conv_paddleocr = similarity_score(real_plate, conv_paddleocr) if conv_paddleocr is not None else 0
        best_score_conv = max(score_conv_easyocr, score_conv_paddleocr)
        
        score_pred_easyocr = similarity_score(real_plate, pred_easyocr) if pred_easyocr is not None else 0
        score_pred_paddleocr = similarity_score(real_plate, pred_paddleocr) if pred_paddleocr is not None else 0
        best_score_pred = max(score_pred_easyocr, score_pred_paddleocr)
        
        total_score = best_score_conv + best_score_pred
        
        if total_score > best_score:
            best_match = real_plate
            best_score = total_score

    return best_match

## Executa o processamento da quantidade de placas reais para o W&B

In [ ]:
def run_real_plates(files, database):
    pre_run = wandb.init(
                project = 'YOLO License Plate - PaddleOCR and EasyOCR',
                name = 'Real'
            )
    
    for file in files:            
        frame_id = os.path.splitext(file)[0]
        
        pre_run.log(
            {
                "model_license_plate_qt": get_frame_data(frame_id, database)['count'],
            },
            commit=True,
        )
            
    pre_run.finish()

## Processamento de detecção e reconhecimento de placas

In [ ]:
def proccess_frames(directory_frames):
    if not os.path.isdir(directory_frames):
        print(f"O diretório {directory_frames} não existe.")
        return
    
    files = os.listdir(directory_frames)
    files.sort()

    database = pd.read_csv('/kaggle/input/datatest/database-plates-types.csv')
    coco_data = load_coco_json('/kaggle/input/datatest/coco_annotations_all_test_frames.json')

    run_real_plates(files, database)

    easeocr_reader   = easyocr.Reader(['en'], gpu=True)
    en_custom_dict = '/kaggle/input/models/my_en_custom_dict_no_point.txt'
    paddleocr_reader = PaddleOCR(use_angle_cls=True, show_log=False, use_gpu=True, rec_char_dict_path=en_custom_dict, lang='en', use_space_char=False)
    
    models = [
        {"model": YOLO('/kaggle/input/models/yolov10s-gb.pt'), "name": 'YOLOv10s-gb'},
        {"model": YOLO('/kaggle/input/models/yolov9s-gb.pt'), "name": 'YOLOv9s-gb'},
        {"model": YOLO('/kaggle/input/models/yolov8s-gb.pt'), "name": 'YOLOv8s-gb'},
        {"model": YOLO('/kaggle/input/models/yolov6s-gb.pt'), "name": 'YOLOv6s-gb'},
        {"model": YOLO('/kaggle/input/models/yolov5su-gb.pt'), "name": 'YOLOv5su-gb'},
        {"model": YOLO('/kaggle/input/models/yolov5su-g.pt'), "name": 'YOLOv5su-g'}
    ]
    
    pbar = tqdm(total=(len(files)*len(models)), desc="Processing")
    
    evals = []
    all_check_hits = []
    count_models = 0
    for model in models:
        count_models += 1

        run = wandb.init(
            project = 'YOLO License Plate Recognition - PaddleOCR and EasyOCR',
            name = model["name"]
        )

        bbox_to_json = []
        license_plate_results = []
        for file in files:            
            frame_id = os.path.splitext(file)[0]
            image_id = frame_id.split('_')[1]
            real_license_plates = get_frame_data(frame_id, database)['plates']
            
            # Load image
            frame_path = os.path.join(directory_frames, (file))
            img_array = cv2.imread(frame_path)
            img_frame = Image.open(frame_path)

            # License plate detection
            results = model["model"](img_frame, save=False, classes=[0], conf=0.25, imgsz=640, device='0', verbose=False)
            
            for result in results:
                confidences = result.boxes.conf.detach().cpu().tolist()
                bboxes = result.boxes.xywhn.cpu().numpy()
                
                unique_bboxes = filter_bboxes_by_max_iou(bboxes, confidences, coco_data, int(image_id))
                
                unique_confidences = []
                for bbox_rescaled, confidence in unique_bboxes:                    
                    iou_valid = check_iou_for_image(coco_data, int(image_id), bbox_rescaled)
                    if iou_valid:
                        unique_confidences.append(confidence)
                        
                        bbox_to_json.append(
                            {
                                "image_id"    : int(image_id),
                                "category_id" : 0,
                                "bbox"        : bbox_rescaled,
                                "score"       : confidence,
                            }
                        )
                        
                        x_left, y_left, width, height = bbox_rescaled
    
                        # Cutting coordinates
                        left = int(x_left)
                        upper = int(y_left)
                        right = int(x_left + width)
                        lower = int(y_left + height)
                        
                        plate_cropped = img_array[upper:lower, left:right]

                        type_plate = get_frame_data(frame_id, database)['type']
                        
                        # Optical character recognition (OCR)
                        conv_easyocr, pred_easyocr = easyocr_get_plate_chars(plate_cropped, easeocr_reader, 0.53)
                        conv_paddleocr, pred_paddleocr = paddle_get_plate_chars(plate_cropped, paddleocr_reader, type_plate)
                        
                        real_license_plate = get_single_real_plate(
                            real_license_plates.split(','), 
                            conv_easyocr, 
                            pred_easyocr,
                            conv_paddleocr, 
                            pred_paddleocr,
                        )
                        
                        license_plate_results.append(
                            {
                                'frame_id': frame_id,
                                'type': type_plate,
                                'locale': get_frame_data(frame_id, database)['locale'],
                                'real_license_plate': real_license_plate,
                                'pred_license_plate': [pred_easyocr, pred_paddleocr],
                                'conv_license_plate': [conv_easyocr, conv_paddleocr],
                            }
                        )
                
                run.log(
                    {
                        "mean_confidence":  np.mean(unique_confidences) if unique_confidences else 0,
                        "model_license_plate_qt": len(unique_confidences) if unique_confidences else 0,
                        "preprocess": result.speed["preprocess"],
                        "inference": result.speed["inference"],
                        "postprocess": result.speed["postprocess"],
                        "total": sum(result.speed.values()),
                    }, 
                    commit=True,
                )
                
            pbar.update(1)        
        
        # Salve data in JSON
        output_json_path = model["name"] + '_results_coco_format'
        with open(output_json_path + '.json', 'w') as f:
            json.dump(bbox_to_json, f, indent=4)
                
        evals.append(
            {
                'model': model["name"],
                'results': model_coco_eval(output_json_path),
            }
        )
        
        char_hit_check_results = all_license_plate_hit_check(license_plate_results)
        
        bar_chart_name = f'chart_hit_check_results_{model["name"]}'
        create_graph_bar(char_hit_check_results, bar_chart_name, model["name"])
        
        all_check_hits.append(
            {
                'model': model["name"],
                'results': char_hit_check_results,
            }
        )
    
        run.log(
            {
                "bar_chart_char_comp_qt": wandb.Image(bar_chart_name + '.png'),
            },
            commit=False,
        )
        
        if count_models == len(models):
            generate_apr_graph(evals)

            create_hit_charts(all_check_hits)

            run.log(
                {
                    "general_apr": wandb.Image('general_apr.png'),
                    "line_chart_fullhit_checks": wandb.Image('line_chart_fullhit_checks.png'),
                },
                commit=False,
            )
        
        run.finish()

    pbar.close()

## Inicia o processamento de imagens

In [ ]:
proccess_frames('/kaggle/input/datatest/ALL_FINAL_TEST_FRAMES')